# Notebook II - Conduct an experiment with PCA-based Proposed Method
Laurine Dargaud, June 2022 (Biometric Systems, DTU course)

**Topic: Face Morphing Attack Detection**

**Parameters to fill in, for training and test dataset:**
- DATABASE: `FERETsub` or `FRGCsub`
- MORPHING ALGO: `facefusion` or `ubo` or `facemorpher` or `opencv`
- size of images, in pixel

**Pre-requisites:**
- create an empty Experiment repository in `data/S-MAD-Experiments/`

In [1]:
# name of experiment repository
_EXPERIMENT_NUMBER_ = '39bis'

# training set parameters
_DATABASE_TRAIN_ = 'FERETsub'
_MORPHING_ALGO_TRAIN_ = 'ubo'

# test set parameters
_DATABASE_TEST_ = 'FRGCsub'
_MORPHING_ALGO_TEST_ = 'ubo'

# size of images
_IMAGE_WIDTH_ = _IMAGE_HEIGHT_ = 500
print('Width x Height =', _IMAGE_WIDTH_, 'x', _IMAGE_HEIGHT_)

Width x Height = 500 x 500


## Global Imports and Definitions

In [2]:
from os import listdir
from os.path import isfile, join, isdir

_EXPERIMENT_PATH_ = f'data/S-MAD-Experiments/{_EXPERIMENT_NUMBER_}/'
print(_EXPERIMENT_PATH_)

all_files_in_exp_repo = [f for f in listdir(_EXPERIMENT_PATH_) if isfile(join(_EXPERIMENT_PATH_, f))]

data/S-MAD-Experiments/39bis/


## Phase 1 - generate TRAINING SET and TEST SET file paths

### Import & Methods

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
def generate_Xy(aDatabaseName, aMorphingAlgo, aTransformation = None, ratioBF = None, ratioM = None, merge_bf_m=True):
    # generate paths
    if aTransformation==None:
        bf_target_path = f'data/{aDatabaseName}-Processed/bona-fide/'
        morph_target_path = f'data/{aDatabaseName}-Processed/morphed-{aMorphingAlgo}/'
    else:
        bf_target_path = f'data/{aDatabaseName}-Processed/bona-fide-{aTransformation}/'
        morph_target_path = f'data/{aDatabaseName}-Processed/morphed-{aMorphingAlgo}-{aTransformation}/'
    print('BF path:', bf_target_path)
    print('Morphed path:', morph_target_path)
    # load feature pics
    bf_features_filenames = [bf_target_path+f for f in listdir(bf_target_path) if isfile(join(bf_target_path, f))]
    m_features_filenames = [morph_target_path+f for f in listdir(morph_target_path) if isfile(join(morph_target_path, f))]
    # apply ratio if needed
    if ratioBF != None:
        if ratioBF < 1:
            bf_features_filenames = random.sample(bf_features_filenames,int(ratioBF*len(bf_features_filenames)))
        else:
            bf_features_filenames = random.sample(bf_features_filenames,int(ratioBF))
    if ratioM != None:
        if ratioM < 1:
            m_features_filenames = random.sample(m_features_filenames,int(ratioM*len(m_features_filenames)))
        else:
            m_features_filenames = random.sample(m_features_filenames,int(ratioM))
    # print sizes
    print('Number of Bona Fide pictures: ', len(bf_features_filenames))
    print('Number of Morphed pictures: ', len(m_features_filenames))
    if merge_bf_m:
        # create merged X and y variables
        X = bf_features_filenames+m_features_filenames
        y = len(bf_features_filenames)*[0]+len(m_features_filenames)*[1]
        return X, y
    return bf_features_filenames, m_features_filenames, len(bf_features_filenames)*[0], len(m_features_filenames)*[1]

### MAIN to run

In [7]:
if (_MORPHING_ALGO_TRAIN_ == _MORPHING_ALGO_TEST_) and (_DATABASE_TRAIN_ == _DATABASE_TEST_):
    # CASE: training and test on the same dataset -> 70/30 split
    _MORPHING_ALGO_ = _MORPHING_ALGO_TRAIN_
    _DATABASE_ = _DATABASE_TRAIN_
    # generate all files
    print('== DATASET ==')
    X, y = generate_Xy(_DATABASE_, _MORPHING_ALGO_)
    # split train and test sets
    X_filenames_train, X_filenames_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    print('\n-- Proportion of positive labels in training set:', np.mean(y_train))
    print('-- Proportion of positive labels in test set:', np.mean(y_test))
    
elif (_DATABASE_TRAIN_ == _DATABASE_TEST_) and (_MORPHING_ALGO_TRAIN_ != _MORPHING_ALGO_TEST_):
    # CASE: Same database but different morphing algorithms
    _MORPHING_ALGO_ = _MORPHING_ALGO_TRAIN_+'>>'+_MORPHING_ALGO_TEST_
    _DATABASE_ = _DATABASE_TRAIN_
    # in this case: X_filenames_train_bf == X_filenames_test_bf & y_train_bf == y_test_bf
    X_filenames_bf, X_filenames_train_m, y_bf, y_train_m = generate_Xy(_DATABASE_TRAIN_, _MORPHING_ALGO_TRAIN_, merge_bf_m=False)
    print('\n')
    _, X_filenames_test_m, _, y_test_m = generate_Xy(_DATABASE_TEST_, _MORPHING_ALGO_TEST_, merge_bf_m=False)
    # SPLIT: 70/30 split for BF, balanced datasets
    # 70/30 for BF
    X_filenames_train_bf, X_filenames_test_bf, y_train_bf, y_test_bf = train_test_split(X_filenames_bf, y_bf, test_size=0.3, random_state=42, shuffle=True)
    # keep 70% of Train morphs (balanced)
    X_filenames_train_m, _, y_train_m, _ = train_test_split(X_filenames_train_m, y_train_m, test_size=0.3, random_state=42, shuffle=True)
    # keep 30% of Test morphs (balanced)
    _, X_filenames_test_m, _, y_test_m = train_test_split(X_filenames_test_m, y_test_m, test_size=0.3, random_state=42, shuffle=True)
    # merge to obtain 2 X_filenames and 2 y lists
    X_filenames_train = list(X_filenames_train_bf) + list(X_filenames_train_m)
    X_filenames_test = list(X_filenames_test_bf) + list(X_filenames_test_m)
    y_train = list(y_train_bf) + list(y_train_m)
    y_test = list(y_test_bf) + list(y_test_m)
    print('\n== TRAINING SET ==')
    print(f'{len(X_filenames_train_bf)} Bona Fide images + {len(X_filenames_train_m)} {_MORPHING_ALGO_TRAIN_} morphed images')
    print('-- Proportion of positive labels:', np.mean(y_train))
    print('\n== TEST SET ==')
    print(f'{len(X_filenames_test_bf)} remaining Bona Fide images + {len(X_filenames_test_m)} {_MORPHING_ALGO_TEST_} morphed images')
    print('-- Proportion of positive labels:', np.mean(y_test))
    
else:
    # CASE: different databases
    _MORPHING_ALGO_ = _MORPHING_ALGO_TRAIN_+'>>'+_MORPHING_ALGO_TEST_
    _DATABASE_ = _DATABASE_TRAIN_+'>>'+_DATABASE_TEST_
    # generate X_filenames_train and y_train
    print('== TRAINING SET ==')
    X_filenames_train, y_train = generate_Xy(_DATABASE_TRAIN_, _MORPHING_ALGO_TRAIN_)
    print('\n-- Proportion of positive labels:', np.mean(y_train))

    # generate X_filenames_test and y_test
    print('\n== TEST SET ==')
    X_filenames_test, y_test = generate_Xy(_DATABASE_TEST_, _MORPHING_ALGO_TEST_)
    print('\n-- Proportion of positive labels:', np.mean(y_test))

== TRAINING SET ==
BF path: data/FERETsub-Processed/bona-fide/
Morphed path: data/FERETsub-Processed/morphed-ubo/
Number of Bona Fide pictures:  622
Number of Morphed pictures:  529

-- Proportion of positive labels: 0.4596003475238923

== TEST SET ==
BF path: data/FRGCsub-Processed/bona-fide/
Morphed path: data/FRGCsub-Processed/morphed-ubo/
Number of Bona Fide pictures:  1274
Number of Morphed pictures:  964

-- Proportion of positive labels: 0.4307417336907953


In [8]:
# save them as txt files in experiment repository

# save filenames
txt_train = ';'.join(X_filenames_train)
txt_test = ';'.join(X_filenames_test)

with open(_EXPERIMENT_PATH_+'X_filenames_train.txt', 'w') as f:
    f.write(txt_train)
with open(_EXPERIMENT_PATH_+'X_filenames_test.txt', 'w') as f:
    f.write(txt_test)
    
# save targets
np.savetxt(_EXPERIMENT_PATH_+'y_train.txt', y_train, delimiter=";")
np.savetxt(_EXPERIMENT_PATH_+'y_test.txt', y_test, delimiter=";")

# save infos
experiment_infos = {
    'training_set_size':len(y_train),
    'test_set_size':len(y_test),
    'nb_morphed_in_training':np.sum(y_train),
    'nb_morphed_in_test':np.sum(y_test),
    'height':_IMAGE_HEIGHT_,
    'width':_IMAGE_WIDTH_,
    'morphing_algo':_MORPHING_ALGO_,
    'dataset':_DATABASE_
}
print(str(experiment_infos))

with open(_EXPERIMENT_PATH_+'infos.txt', 'w') as f:
    f.write(str(experiment_infos))

{'training_set_size': 1151, 'test_set_size': 2238, 'nb_morphed_in_training': 529, 'nb_morphed_in_test': 964, 'height': 500, 'width': 500, 'morphing_algo': 'ubo>>ubo', 'dataset': 'FERETsub>>FRGCsub'}


## Phase 2 - run PCA for each color channel

### Imports & Methods

In [9]:
import cv2
from sklearn.decomposition import PCA
import numpy as np
from joblib import dump

In [10]:
def process_image(anImage):
    return cv2.cvtColor(anImage, cv2.COLOR_BGR2GRAY)

def get_data_matrix(aXfilenames, H = _IMAGE_HEIGHT_, W = _IMAGE_WIDTH_):
    # get the number of pictures in the training set
    N = len(aXfilenames)

    # get height and width of image
    M = H*W

    # create N*M data matrix of zeros
    data = np.zeros((M,N))

    # fill the empty matrix so that each column is one face image
    for k in range (N):
        # read image
        img = cv2.imread(aXfilenames[k])
        # resize
        if img.shape[:2] != (W,H):
            img = cv2.resize(img, (W,H))
        # process image
        img = process_image(img)
        # reshape
        tt = np.reshape(img, M)
        # fill data matrix
        data[:,k] = tt

    X = data.T
    return X

def change_color_channel_in_path(aPath, aColorChannel):
    result = aPath.split('/')
    result[-2] = result[-2]+f'-lpb-{aColorChannel}comp-cropped'
    return '/'.join(result)

### MAIN to run

In [11]:
_NB_COMPONENTS_ = 20

In [12]:
# load txt files from experiment
with open(_EXPERIMENT_PATH_+'X_filenames_train.txt') as f:
    X_filenames_train = f.readlines()[0]
    X_filenames_train = X_filenames_train.split(';')

with open(_EXPERIMENT_PATH_+'X_filenames_test.txt') as f:
    X_filenames_test = f.readlines()[0]
    X_filenames_test = X_filenames_test.split(';')
    
y_train = np.loadtxt(_EXPERIMENT_PATH_+'y_train.txt', delimiter=';')
y_test = np.loadtxt(_EXPERIMENT_PATH_+'y_test.txt', delimiter=';')

In [13]:
for aColorChannel in ['R', 'G', 'B']:
    if f'X_train_transformed_{aColorChannel}.txt' not in all_files_in_exp_repo:
        # generate path of color channel features repo
        X_filenames_train_color = list(map(lambda p: change_color_channel_in_path(p, aColorChannel), X_filenames_train))
        X_filenames_test_color = list(map(lambda p: change_color_channel_in_path(p, aColorChannel), X_filenames_test))
        # run PCA
        print(f'Running PCA for {aColorChannel}...')
        X_train = get_data_matrix(X_filenames_train_color)
        pca = PCA(_NB_COMPONENTS_).fit(X_train)
        # save pca
        dump(pca, _EXPERIMENT_PATH_+f'pca{aColorChannel}.joblib') 
        # project training set into PC space
        print('  Projecting X_train into PC space...')
        X_train_transformed = pca.transform(X_train)
        # project test set into PC space
        print('  Projecting X_test into PC space...')
        X_test = get_data_matrix(X_filenames_test_color)
        X_test_transformed = pca.transform(X_test)
        # save PC 2D coordinates of training dataset
        print('  Saving results...\n')
        np.savetxt(_EXPERIMENT_PATH_+f'X_train_transformed_{aColorChannel}.txt', X_train_transformed, delimiter=";")
        # save PC 2D coordinates of test dataset
        np.savetxt(_EXPERIMENT_PATH_+f'X_test_transformed_{aColorChannel}.txt', X_test_transformed, delimiter=";")

Running PCA for R...
  Projecting X_train into PC space...
  Projecting X_test into PC space...
  Saving results...

Running PCA for G...
  Projecting X_train into PC space...
  Projecting X_test into PC space...
  Saving results...

Running PCA for B...
  Projecting X_train into PC space...
  Projecting X_test into PC space...
  Saving results...



## Phase 3 - run Bayesian Classification for each color channel + run final classification RF

### Imports & Methods

In [14]:
import autograd.numpy as np

from exercise5 import compute_err
from exercise5 import generate_samples
from exercise5 import GaussianProcessModel

from sklearn.ensemble import RandomForestClassifier

In [15]:
thresold = 0.5

# fix random seed
np.random.seed(0)

sigmoid = lambda x: 1./(1+np.exp(-x))
y = lambda x:  5*np.sin(0.75*x)

def log_lik_bernoulli(y, t): 
    """ implement log p(t=1|y) using the sigmoid inverse link function """
    p = sigmoid(y)
    return t.ravel()*np.log(p) + (1-t.ravel())*np.log(1-p)

def compute_predictive_prob_MC(mu_y, Sigma_y, sample_size=2000):
    """
        The function computes p(t^* = 1|t, x^*) using Monte Carlo sampling  as in eq. (2).
        The function also returns the samples generated in the process for plotting purposes

        arguments:
        mu_y             -- N x 1 vector
        Sigma_y          -- N x N matrix
        sample_size      -- positive integer

        returns:
        p                -- N   vector
        y_samples        -- sample_size x N matrix
        sigma_samples    -- sample_size x N matrix

    """

    # generate samples from y ~ N(mu, Sigma)
    y_samples = generate_samples(mu_y, Sigma_y, sample_size).T 

    # apply inverse link function (elementwise)
    sigma_samples = sigmoid(y_samples)

    # return MC estimate, samples of y and sigma(y)
    return np.mean(sigma_samples, axis=0), y_samples, sigma_samples

def predictive(Xp, aGP):
    mu_y, var_y = aGP.compute_posterior_y(Xp, pointwise=True)
    p, _, _ = compute_predictive_prob_MC(mu_y, np.diag(var_y))
    return p

def run_classification(related_channel_short):
    print(f'Classification for {related_channel_short} color channel:')
    # load files
    Xtrain = np.loadtxt(_EXPERIMENT_PATH_+f'X_train_transformed_{related_channel_short}.txt', delimiter=';')
    Xtest = np.loadtxt(_EXPERIMENT_PATH_+f'X_test_transformed_{related_channel_short}.txt', delimiter=';')
    ttrain = np.loadtxt(_EXPERIMENT_PATH_+'y_train.txt', delimiter=';')
    ttest = np.loadtxt(_EXPERIMENT_PATH_+'y_test.txt', delimiter=';')

    # make sure dimensions are [N x 1]
    ttrain = ttrain[:, None]
    ttest = ttest[:, None]

    # scale to proper dimensions!!
    if related_channel_short != 'RGB':
        scale = 0.0001
        Xtrain, Xtest = scale*Xtrain, scale*Xtest

    X = np.row_stack((Xtrain, Xtest))
    t = np.row_stack((ttrain, ttest))

    # define prior
    kappa = 1.
    scale = 1.
    theta = [kappa, scale]

    # fit classifier and prediction
    if related_channel_short == 'RGB':
        print('     Running Random Forest...')
        best_classifier = 'Random Forest'
        clf = RandomForestClassifier(max_depth=2, random_state=42)
        clf.fit(Xtrain, ttrain.ravel())
        print('     Predicting...')
        p_train = clf.predict_proba(Xtrain).T[1]
        p_test = clf.predict_proba(Xtest).T[1]
    else:
        print('     Running Gaussian Process model...')
        best_classifier = 'Gaussian Process'
        gp = GaussianProcessModel(Xtrain, ttrain, theta, log_lik_bernoulli)
        print('     Predicting...')
        p_train = predictive(Xtrain, gp)
        p_test = predictive(Xtest, gp)

    # make predictions
    ttrain_hat = 1.0*(p_train > thresold)
    ttest_hat = 1.0*(p_test > thresold)

    # print results: mean and standard error of the mean
    print('     - Training error:\t%3.2f (%3.2f)' % compute_err(ttrain_hat.ravel(), ttrain.ravel()))
    print('     - Test error:\t%3.2f (%3.2f)' % compute_err(ttest_hat.ravel(), ttest.ravel()))

    # compute performance scores
    NFCM = np.sum(ttest.ravel() > ttest_hat.ravel())
    MFCN = np.sum(ttest.ravel() < ttest_hat.ravel())
    NFCM = NFCM/(len(ttest.ravel())-np.sum(ttest.ravel()))
    MFCN = MFCN/(np.sum(ttest.ravel()))
    ACER = (NFCM+MFCN)/2

    print(f'         NFCM = {round(NFCM*100,3)}%')
    print(f'         MFCN = {round(MFCN*100,3)}%')
    print(f'         >>>> ACER = {round(ACER*100,3)}%')

    # export results
    performance_results = {
        'NFCM':NFCM,
        'MFCN':MFCN,
        'ACER':ACER,
        'classifier':best_classifier,
        'threshold':thresold
    }

    with open(_EXPERIMENT_PATH_+f'results_{related_channel_short}.txt', 'w') as f:
        f.write(str(performance_results))

    # export probas
    np.savetxt(_EXPERIMENT_PATH_+f'y_train_probas_{related_channel_short}.txt', p_train, delimiter=";")
    np.savetxt(_EXPERIMENT_PATH_+f'y_test_probas_{related_channel_short}.txt', p_test, delimiter=";")

### MAIN to run

In [16]:
for related_channel_short in ['R','G','B']:
    
    if f'y_test_probas_{related_channel_short}.txt' not in all_files_in_exp_repo:
        run_classification(related_channel_short)

# create data
if f'X_train_transformed_RGB.txt' not in all_files_in_exp_repo:
    X_train_transformed, X_test_transformed = [], []

    for aSymbol in ['R','G','B']:
        # get y_train_probas
        y_train_probas = np.loadtxt(_EXPERIMENT_PATH_+f'y_train_probas_{aSymbol}.txt', delimiter=';')
        # get y_test_probas
        y_test_probas = np.loadtxt(_EXPERIMENT_PATH_+f'y_test_probas_{aSymbol}.txt', delimiter=';')
        X_train_transformed.append(y_train_probas)
        X_test_transformed.append(y_test_probas)

    X_train_transformed = np.array(X_train_transformed).T
    X_test_transformed = np.array(X_test_transformed).T

    # save PC 2D coordinates of training dataset
    np.savetxt(_EXPERIMENT_PATH_+f'X_train_transformed_RGB.txt', X_train_transformed, delimiter=";")

    # save PC 2D coordinates of test dataset
    np.savetxt(_EXPERIMENT_PATH_+f'X_test_transformed_RGB.txt', X_test_transformed, delimiter=";")

# run final classification
run_classification('RGB')

Classification for R color channel:
     Running Gaussian Process model...
     Predicting...
     - Training error:	0.14 (0.01)
     - Test error:	0.14 (0.01)
         NFCM = 6.515%
         MFCN = 24.793%
         >>>> ACER = 15.654%
Classification for G color channel:
     Running Gaussian Process model...
     Predicting...
     - Training error:	0.13 (0.01)
     - Test error:	0.11 (0.01)
         NFCM = 16.876%
         MFCN = 2.697%
         >>>> ACER = 9.787%
Classification for B color channel:
     Running Gaussian Process model...
     Predicting...
     - Training error:	0.11 (0.01)
     - Test error:	0.13 (0.01)
         NFCM = 21.9%
         MFCN = 0.207%
         >>>> ACER = 11.053%
Classification for RGB color channel:
     Running Random Forest...
     Predicting...
     - Training error:	0.11 (0.01)
     - Test error:	0.10 (0.01)
         NFCM = 16.641%
         MFCN = 1.141%
         >>>> ACER = 8.891%


## Phase 4 - create mated and nonmated txt files for DET script

### Imports & Methods

In [17]:
import numpy as np

### MAIN to run

In [18]:
y_test = np.loadtxt(_EXPERIMENT_PATH_+'y_test.txt', delimiter=';')

y_test_morphedIdx = np.argwhere(y_test == 1).ravel()
y_test_bonafideIdx = np.argwhere(y_test == 0).ravel() 

for aColorChannel in ['R','G','B','RGB']:
    y_test_probas = np.loadtxt(_EXPERIMENT_PATH_+f'y_test_probas_{aColorChannel}.txt', delimiter=';')
    y_test_mated_probas = y_test_probas[y_test_bonafideIdx]
    y_test_nonmated_probas = y_test_probas[y_test_morphedIdx]
    # save files
    np.savetxt(_EXPERIMENT_PATH_+f'mated{aColorChannel}.txt', y_test_mated_probas, delimiter="\t")
    np.savetxt(_EXPERIMENT_PATH_+f'nonmated{aColorChannel}.txt', y_test_nonmated_probas, delimiter="\t")

# Generate reports

In [3]:
def file2dict(aPath):
    with open(aPath) as f:
        result = f.readlines()[0]
        result = ast.literal_eval(result)
    return result

def dict_report(aDict, title):
    result = '>> '+title + '\n'
    for k,v in aDict.items():
        result += f'\t{k}: {v}\n'
    return result+'\n'

import ast

_GLOBAL_EXPERIMENT_PATH_ = 'data/S-MAD-Experiments/'

experiments_directories = [f for f in listdir(_GLOBAL_EXPERIMENT_PATH_) if isdir(join(_GLOBAL_EXPERIMENT_PATH_, f))]
print('Experiments:',experiments_directories)

final_report = ''

# generate report
for anExp in experiments_directories:
    final_report = f'=== EXPERIMENT {anExp} ===\n\n'
    path = _GLOBAL_EXPERIMENT_PATH_+str(anExp)+'/'
    # all files in the experiment repo
    allFiles = [f for f in listdir(path) if isfile(join(path, f))]
    # read notes
    if 'notes.txt' in listdir(path):
        with open(path+'notes.txt') as f:
            notes = f.readlines()
            final_report += '* NOTES *\n'+''.join(notes)+'\n\n'
    # read infos
    expInfos = file2dict(path+'infos.txt')
    final_report += dict_report(expInfos, 'INFOS')
    all_results_filenames = {
        'B':'Blue Component','G':'Green Component', 'R':'Red Component','Gray':'Grayscale',
        'H':'Hue Component','S':'Saturation Component', 'V':'Value Component'
    }
    # read results
    for k,v in all_results_filenames.items():
        if (f'results_{k}.txt' in allFiles):
            results = file2dict(path+f'results_{k}.txt')
            final_report += dict_report(results, f'{v} Results')
    # export
    with open(_GLOBAL_EXPERIMENT_PATH_+f'reportExp{anExp}.txt', 'w') as f:
        f.write(final_report)

# merge all reports into one final report
final_report = ''
experiments_reports_files= [_GLOBAL_EXPERIMENT_PATH_+f for f in listdir(_GLOBAL_EXPERIMENT_PATH_) if (isfile(join(_GLOBAL_EXPERIMENT_PATH_, f)) and 'reportExp' in f)]

for aReport in experiments_reports_files:
        with open(aReport) as f:
            notes = f.readlines()
            final_report += ''.join(notes)+'\n\n'

with open(_GLOBAL_EXPERIMENT_PATH_+f'MERGED-REPORT.txt', 'w') as f:
        f.write(final_report)

Experiments: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '37bis', '38', '39', '39bis', '40', '40bis', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71']


# THE END